In [1]:
# !pip3 install malaya -U --no-deps
# !pip3 install unidecode dateparser ftfy herpetologist networkx==2.5.1 sklearn sentencepiece
# !wget https://raw.githubusercontent.com/google-research-datasets/Taskmaster/master/TM-2-2020/data/music.json

In [5]:
import malaya
import json
from tqdm import tqdm
import copy
import utils
import subprocess

In [3]:
import os

files = ['music.json', 'restaurant-search.json', 'sports.json']
for f in files:
    print(f)
    # os.system(f'wget https://raw.githubusercontent.com/google-research-datasets/Taskmaster/master/TM-2-2020/data/{f}')

music.json
restaurant-search.json
sports.json


In [4]:
transformer = malaya.translation.en_ms.transformer()

2021-10-03 22:38:40.364566: I tensorflow/core/platform/cpu_feature_guard.cc:142] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX512F FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2021-10-03 22:38:40.368937: I tensorflow/stream_executor/platform/default/dso_loader.cc:53] Successfully opened dynamic library libcuda.so.1
2021-10-03 22:38:40.425985: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:937] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2021-10-03 22:38:40.427873: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1733] Found device 0 with properties: 
pciBusID: 0000:01:00.0 name: GeForce RTX 3080 computeCapability: 8.6
coreClock: 1.74GHz coreCount: 68 deviceMemorySize: 9.78GiB deviceMemoryBandwidth: 707.88GiB/s
2021-10-

In [7]:
os.path.splitext(files[0])[0]

('music', '.json')

In [8]:
for file in files:
    with open(file) as fopen:
        data = json.load(fopen)
    print(file, len(data))
    
    texts = []
    for row in tqdm(data):
        for u in row['utterances']:
            texts.append(u['text'])
            if 'segments' in u:
                for s in u['segments']:
                    texts.append(s['text'])
    texts = list(set(texts))
    print(file, len(texts))
    
    results = {}
    batch_size = 10
    for i in tqdm(range(0, len(texts), batch_size)):
        b = texts[i: i + batch_size]
        b_ = transformer.greedy_decoder(b)
        for k in range(len(b)):
            results[b[k]] = b_[k]
            
    with open('left', 'w') as fopen:
        fopen.write('\n'.join(list(results.keys())))

    with open('right', 'w') as fopen:
        fopen.write('\n'.join(list(results.values())))
        
    cmd = 'python3 align.py -s left -t right --priors align.priors --model 3 -f out.fwd -r out.rev --overwrite'
    subprocess.run(cmd.split())
    
    with open('out.fwd') as fopen:
        alignment = fopen.read().split('\n')[:-1]
    len(alignment)

    results_alignment = {}
    for no, k in enumerate(results.keys()):
        results_alignment[k + results[k]] = alignment[no]
    
    with open('out.rev') as fopen:
        alignment = fopen.read().split('\n')[:-1]
    len(alignment)

    results_alignment_rev = {}
    for no, k in enumerate(results.keys()):
        results_alignment_rev[k + results[k]] = alignment[no]
    
    data_ = copy.deepcopy(data)
    
    answers_from_alignment = True

    rs = []
    for i in tqdm(range(len(data_))):
        try:
            for u in range(len(data_[i]['utterances'])):
                data_[i]['utterances'][u]['text_ms'] = results[data_[i]['utterances'][u]['text']]
                if 'segments' in data_[i]['utterances'][u]:
                    for k in range(len(data_[i]['utterances'][u]['segments'])):
                        answer = {'text': data_[i]['utterances'][u]['segments'][k]['text'], 
                                  'answer_start': data_[i]['utterances'][u]['segments'][k]['start_index']}
                        answer_translated, answer_translated_start = utils.extract_answer_translated(
                            answer,
                            results[data_[i]['utterances'][u]['segments'][k]['text']],
                            data_[i]['utterances'][u]['text'],
                            results[data_[i]['utterances'][u]['text']],
                            results_alignment[data_[i]['utterances'][u]['text'] + results[data_[i]['utterances'][u]['text']]],
                            answers_from_alignment,
                        )
                        data_[i]['utterances'][u]['segments'][k]['text_ms'] = answer_translated
                        data_[i]['utterances'][u]['segments'][k]['start_index_ms'] = answer_translated_start
                        data_[i]['utterances'][u]['segments'][k]['end_index_ms'] = answer_translated_start + len(answer_translated)
                        r = results[data_[i]['utterances'][u]['text']]
                        if r[answer_translated_start: answer_translated_start + len(answer_translated)] == results[data_[i]['utterances'][u]['segments'][k]['text']]:
                            data_[i]['utterances'][u]['segments'][k]['aligned'] = True
                        else:
                            data_[i]['utterances'][u]['segments'][k]['aligned'] = False
            rs.append(data_[i])
        except Exception as e:
            print(e)
            pass
    
    with open(f'{os.path.splitext(file)[0]}.json', 'w') as fopen:
        json.dump(rs, fopen)

music.json 1603


100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 1603/1603 [00:00<00:00, 378851.03it/s]


music.json 27454


  0%|                                                                                                                                                             | 0/2746 [00:00<?, ?it/s]2021-10-03 22:41:01.213964: I tensorflow/core/platform/profile_utils/cpu_utils.cc:114] CPU Frequency: 2496000000 Hz
2021-10-03 22:41:04.902944: I tensorflow/stream_executor/platform/default/dso_loader.cc:53] Successfully opened dynamic library libcublas.so.11
2021-10-03 22:41:05.223558: I tensorflow/stream_executor/platform/default/dso_loader.cc:53] Successfully opened dynamic library libcublasLt.so.11
2021-10-03 22:41:05.223590: I tensorflow/stream_executor/cuda/cuda_blas.cc:1838] TensorFloat-32 will be used for the matrix multiplication. This will only be logged once.
100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 1603/1603 [00:00<00:00, 1852.71it/s]


restaurant-search.json 3276


...

100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 3276/3276 [00:03<00:00, 1054.54it/s]


sports.json 3481


100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 3481/3481 [00:00<00:00, 274164.80it/s]


sports.json 38452


100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 3481/3481 [00:02<00:00, 1635.26it/s]
